In [210]:
%matplotlib widget

In [211]:
import camelot
from pathlib import Path
import pandas as pd
import numpy as np

In [212]:
src_pdf_filename = "march_2022"
payroll_report_pdf = Path.cwd().parent / "data" / "raw" / "payroll_reports" / f"{src_pdf_filename}.pdf"

In [213]:
raw_tables = camelot.read_pdf(str(payroll_report_pdf), pages="all", flavor="lattice")

/usr/local/lib/python3.10/dist-packages/camelot/utils.py:740: UserWarning:   (560.9875199999999, 563.4825599999999) does not lie in column range (46.78963100816204, 560.0471100362757)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/camelot/utils.py:740: UserWarning:   (560.97648, 563.47152) does not lie in column range (46.79009966369408, 560.5270133010883)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/camelot/utils.py:740: UserWarning:   (561.0, 563.49504) does not lie in column range (46.79009966369408, 560.5270133010883)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/camelot/utils.py:740: UserWarning:   (451.4265600000001, 453.9216000000001) does not lie in column range (46.78963100816204, 451.3490205562274)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/camelot/utils.py:740: UserWarning:   (451.4155200000002, 453.9105600000002) does not lie in column range (46.79009966369408, 451.10906892382104)
  warnings.warn(
/usr/local/lib/python3.10/dist

In [214]:
tables = list(map(lambda t: t.df.copy(), raw_tables))

In [215]:
def basic_cleaning(df: pd.DataFrame):
    df = df.dropna(how="all", axis=1)
    df = df.replace(r"[\r\n\s]+", " ", regex=True) \
        .replace('-', '', regex=True) \
        .replace("", np.nan)
    print("Load Table:", df.shape, "->", df.shape)
    return df

In [216]:
cln_tables = list(map(basic_cleaning, tables))

Load Table: (9, 16) -> (9, 16)
Load Table: (9, 16) -> (9, 16)
Load Table: (9, 16) -> (9, 16)
Load Table: (9, 16) -> (9, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (9, 9) -> (9, 9)
Load Table: (9, 9) -> (9, 9)
Load Table: (9, 9) -> (9, 9)
Load Table: (9, 9) -> (9, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> 

In [217]:
def is_monthly(df: pd.DataFrame):
    return df.iloc[0, :].str.contains("\w{3,9}\s\d{4}", regex=True).any()

In [218]:
def is_epf(df: pd.DataFrame):
    return df.iloc[:, 1].str.lower().str.contains('new EPF subscriber', na=False, case=False).any()

In [219]:
monthly_epf_tables = list(filter(is_monthly, filter(is_epf, cln_tables)))

In [220]:
def parse_date_head(df: pd.DataFrame) -> pd.Series:
    return pd.to_datetime(
        df.iloc[:, 0],
        format="%B %Y",
        errors="coerce"
    ).dropna()

In [221]:
def strip_month_headline(df: pd.DataFrame):
    date_df = parse_date_head(df)
    date_row_idx = date_df.index[0]
    return df.iloc[date_row_idx + 1:].reset_index(drop=True)


In [222]:
def exclude_totals_row(df: pd.DataFrame):
    return df[~df.iloc[:, 0].str.contains("total", na=False, case=False)]

In [223]:
def epf_correct_camelot(df: pd.DataFrame):
    # TODO: find a method to resolve this issue with camelot itself.
    if df.iloc[:, 0].str.match(r'([\s\w]+) (\d) (\d[\d,.]*)$').any():
        df.iloc[7, [0,1]] = df.iloc[:, 0].str.extract(r'(.*) (\d[\d,.]*)$').iloc[7]
        df.iloc[2, [0,1]] = df.iloc[:, 1].str.extract(r'(.*) (\d[\d,.]*)$').iloc[2]
    return df

In [224]:
def prep_row_labels(df: pd.DataFrame, lheads=["head", "gender"]):
    headings = df.iloc[:, 0].str.replace(r"(\d{2})(\d{2})", r"\1-\2", regex=True)
    headings[0:len(lheads)] = lheads
    headings = headings.str.lower()
    return df.rename(index=headings).drop(0, axis=1)

In [225]:
def reshape_epf(df: pd.DataFrame, id_vars=["head", "gender"]):
    df.iloc[0] = df.iloc[0].str.replace("\n", "").replace("", np.nan).ffill()
    df = df.T.melt(id_vars=id_vars, var_name="age")
    categorical_columns = id_vars + ["age"]
    df[categorical_columns] = df[categorical_columns].astype("category")
    # TODO: convert values to integer values
    # df.value = pd.to_numeric(df.value.str.replace(",", "").str.strip(), errors="coerce", downcast="unsigned")
    return df

In [226]:
def display_df(df: pd.DataFrame):
    display(df)
    return df

In [227]:
monthly_epf_tables[0].pipe(strip_month_headline).pipe(exclude_totals_row).pipe(display_df).pipe(epf_correct_camelot).pipe(prep_row_labels).pipe(display_df).pipe(reshape_epf)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Age,Number of new EPF subscribers during the month,NaN,NaN,NaN,NaN,Number of members that ceased subscribing duri...,NaN,NaN,NaN,NaN,Number of exited members who rejoined and resu...,NaN,NaN,NaN,NaN
1,NaN,Male,Female,Others,Not Available,Total,Male,Female,Others,Not Available,Total,Male,Female,Others,Not Available,Total
2,Less than 18,"5,113","2,253",NaN,2,"7,368","1,381",961,NaN,1,"2,343",583,413,NaN,NaN,996
3,1821,"1,59,264","45,618",1,2,"2,04,885","1,25,257","25,808",1,2,"1,51,068","91,712","18,738",NaN,NaN,"1,10,450"
4,2225,"1,51,502","60,961",2,2,"2,12,467","2,44,727","59,732",4,6,"3,04,469","2,32,357","51,407",3,NaN,"2,83,767"
5,2628,"66,699","24,307",1,3,"91,010","1,68,592","39,945",2,26,"2,08,565","1,74,474","36,473",1,NaN,"2,10,948"
6,2935,"93,449","42,673",1,4,"1,36,127","2,50,978","60,885",5,148,"3,12,016","2,60,081","54,954",5,NaN,"3,15,040"
7,More than 35,"1,09,095","44,920",2,4,"1,54,021","2,67,823","75,393",7,309,"3,43,532","2,38,353","55,431",5,1,"2,93,790"


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
head,Number of new EPF subscribers during the month,NaN,NaN,NaN,NaN,Number of members that ceased subscribing duri...,NaN,NaN,NaN,NaN,Number of exited members who rejoined and resu...,NaN,NaN,NaN,NaN
gender,Male,Female,Others,Not Available,Total,Male,Female,Others,Not Available,Total,Male,Female,Others,Not Available,Total
less than 18,"5,113","2,253",NaN,2,"7,368","1,381",961,NaN,1,"2,343",583,413,NaN,NaN,996
18-21,"1,59,264","45,618",1,2,"2,04,885","1,25,257","25,808",1,2,"1,51,068","91,712","18,738",NaN,NaN,"1,10,450"
22-25,"1,51,502","60,961",2,2,"2,12,467","2,44,727","59,732",4,6,"3,04,469","2,32,357","51,407",3,NaN,"2,83,767"
26-28,"66,699","24,307",1,3,"91,010","1,68,592","39,945",2,26,"2,08,565","1,74,474","36,473",1,NaN,"2,10,948"
29-35,"93,449","42,673",1,4,"1,36,127","2,50,978","60,885",5,148,"3,12,016","2,60,081","54,954",5,NaN,"3,15,040"
more than 35,"1,09,095","44,920",2,4,"1,54,021","2,67,823","75,393",7,309,"3,43,532","2,38,353","55,431",5,1,"2,93,790"


,head,gender,age,value
0,Number of new EPF subscribers during the month,Male,less than 18,"5,113"
1,Number of new EPF subscribers during the month,Female,less than 18,"2,253"
2,Number of new EPF subscribers during the month,Others,less than 18,NaN
3,Number of new EPF subscribers during the month,Not Available,less than 18,2
4,Number of new EPF subscribers during the month,Total,less than 18,"7,368"
...,...,...,...,...
85,Number of exited members who rejoined and resu...,Male,more than 35,"2,38,353"
86,Number of exited members who rejoined and resu...,Female,more than 35,"55,431"
87,Number of exited members who rejoined and resu...,Others,more than 35,5
88,Number of exited members who rejoined and resu...,Not Available,more than 35,1


In [228]:
parse_date_head(monthly_epf_tables[0]).iloc[0]

Timestamp('2021-04-01 00:00:00')

In [229]:
def epf_data_pipeline(df: pd.DataFrame):
    month = parse_date_head(df).iloc[0]
    df = df.pipe(strip_month_headline) \
        .pipe(exclude_totals_row) \
        .pipe(epf_correct_camelot) \
        .pipe(prep_row_labels) \
        .pipe(reshape_epf)
    df["year"] = month.year
    df["month"] = month.month
    df["sector"] = np.nan
    return df

In [230]:
fnl_epf = pd.concat(map(epf_data_pipeline, monthly_epf_tables))

In [231]:
def is_esic(df: pd.DataFrame):
    return df.iloc[:, 5].str.lower().str.contains('Number of newly registered', na=False, case=False).any()

In [232]:
monthly_esic_tables = list(filter(is_monthly, filter(is_esic, tables)))

In [233]:
monthly_esic_tables[0].pipe(strip_month_headline).pipe(exclude_totals_row).pipe(prep_row_labels).pipe(display_df).pipe(reshape_epf).head()

,1,2,3,4,5,6,7,8
head,Number of existing employees who paid during t...,,,,Number of newly registered employees & \npayin...,,,
gender,Male,Female,Others,Total,Male,Female,Others,Total
less than 18,"3,697","3,293",,"6,990","1,287","1,179",,"2,466"
18-21,"6,59,718","1,37,628",16,"7,97,362","1,64,651","33,390",1,"1,98,042"
22-25,"31,30,525","7,33,868",105,"38,64,498","2,52,353","60,649",7,"3,13,009"
26-28,"29,35,539","5,64,793",92,"35,00,424","1,30,367","27,451",3,"1,57,821"
29-35,"53,87,458","10,56,183",202,"64,43,843","1,63,346","44,383",8,"2,07,737"
more than 35,"77,96,460","21,94,719",420,"99,91,599","1,53,023","46,333",11,"1,99,367"


,head,gender,age,value
0,Number of existing employees who paid during t...,Male,less than 18,"3,697"
1,Number of existing employees who paid during t...,Female,less than 18,"3,293"
2,Number of existing employees who paid during t...,Others,less than 18,
3,Number of existing employees who paid during t...,Total,less than 18,"6,990"
4,Number of newly registered employees & paying ...,Male,less than 18,"1,287"


In [234]:
def esic_data_pipeline(df: pd.DataFrame):
    month = parse_date_head(df).iloc[0]
    df = df.pipe(strip_month_headline) \
        .pipe(exclude_totals_row) \
        .pipe(prep_row_labels) \
        .pipe(reshape_epf)
    df["year"] = month.year
    df["month"] = month.month
    df["sector"] = np.nan
    return df

In [235]:
fnl_esic = pd.concat(map(esic_data_pipeline, monthly_esic_tables))

In [236]:
fnl_esic

,head,gender,age,value,year,month,sector
0,Number of existing employees who paid during t...,Male,less than 18,"3,697",2021,4,NaN
1,Number of existing employees who paid during t...,Female,less than 18,"3,293",2021,4,NaN
2,Number of existing employees who paid during t...,Others,less than 18,,2021,4,NaN
3,Number of existing employees who paid during t...,Total,less than 18,"6,990",2021,4,NaN
4,Number of newly registered employees & paying ...,Male,less than 18,"1,287",2021,4,NaN
...,...,...,...,...,...,...,...
43,Number of existing employees who paid during t...,Total,more than 35,"1,00,74,164",2022,3,NaN
44,Number of newly registered employees & paying ...,Male,more than 35,"1,94,763",2022,3,NaN
45,Number of newly registered employees & paying ...,Female,more than 35,"63,191",2022,3,NaN
46,Number of newly registered employees & paying ...,Others,more than 35,11,2022,3,NaN


In [237]:
def is_nps(df: pd.DataFrame):
    return df.iloc[:, 1].str.lower().str.replace("\n", "").str.contains('existing subscribers', na=False,case=False).any()


In [238]:
monthly_nps_tables = list(filter(is_monthly, filter(is_nps, tables)))

In [239]:
from functools import partial
prep_nps_row_labels = partial(prep_row_labels, lheads=["head", "sector", "gender"])
# reshape_nps = partial(reshape_epf, id_vars=["head", "gender", "sector"])

In [240]:
def reshape_nps(df: pd.DataFrame, id_vars=["head", "gender", "sector"]):
    df = df.T.replace("", np.nan).ffill()
    df = df.melt(id_vars=id_vars, var_name="age")
    categorical_columns = id_vars + ["age"]
    df[categorical_columns] = df[categorical_columns].astype("category")
    return df

In [241]:
monthly_nps_tables[0].pipe(strip_month_headline).pipe(exclude_totals_row).pipe(prep_nps_row_labels).pipe(display_df).pipe(reshape_nps)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
head,Total Existing \nSubscribers \ncontributing du...,New Subscribers contributing during the month,,,,,,,,,,,,,,,
sector,,Central Govt,,,,,State Govt,,,,,Non-Govt (Corporate Sector),,,,,Total New \nSubscribers
gender,,Male,Female,Transgender,Non-IRA,Total,Male,Female,Transgender,Non-IRA,Total,Male,Female,Transgender,Non-IRA,Total,
18-21,"15,807",210,35,-,,- 245,607,130,-,,- 737,33,39,-,,- 72,"1,054"
22-25,"2,55,214","2,098",312,-,,"- 2,410","3,522","1,586",-,,"- 5,108","1,190",846,-,,"- 2,036","9,554"
26-28,"5,45,355","2,362",376,-,,"- 2,738","4,078","2,329",-,,"- 6,407","1,404",745,-,,"- 2,149","11,294"
29-35,"20,57,265","2,533",606,-,,"- 3,139","10,052","6,240",-,,"- 16,292","4,812","1,035",-,,"- 5,847","25,278"
> 35,"25,76,417",547,204,-,,- 751,"3,054","1,860",-,,"- 4,914","2,620",379,-,,"- 2,999","8,664"
non-ira,"1,720",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


,head,gender,sector,age,value
0,Total Existing \nSubscribers \ncontributing du...,NaN,NaN,18-21,"15,807"
1,New Subscribers contributing during the month,Male,Central Govt,18-21,210
2,New Subscribers contributing during the month,Female,Central Govt,18-21,35
3,New Subscribers contributing during the month,Transgender,Central Govt,18-21,-
4,New Subscribers contributing during the month,Non-IRA,Central Govt,18-21,-
...,...,...,...,...,...
97,New Subscribers contributing during the month,Female,Non-Govt (Corporate Sector),non-ira,-
98,New Subscribers contributing during the month,Transgender,Non-Govt (Corporate Sector),non-ira,-
99,New Subscribers contributing during the month,Non-IRA,Non-Govt (Corporate Sector),non-ira,-
100,New Subscribers contributing during the month,Total,Non-Govt (Corporate Sector),non-ira,-


In [242]:
def nps_data_pipeline(df: pd.DataFrame):
    month = parse_date_head(df).iloc[0]
    df = df.pipe(strip_month_headline) \
        .pipe(exclude_totals_row) \
        .pipe(prep_nps_row_labels) \
        .pipe(reshape_nps)
    df["year"] = month.year
    df["month"] = month.month
    for col in ["head", 'gender', 'sector']:
        df[col] = df[col].str.replace("\n", "")
    df["age"] = df["age"].str.replace("> 35", "more than 35")
    return df

In [243]:
fnl_nps = pd.concat(map(nps_data_pipeline, monthly_nps_tables))

In [244]:
fnl_nps

,head,gender,sector,age,value,year,month
0,Total Existing Subscribers contributing during...,NaN,NaN,18-21,"15,807",2021,4
1,New Subscribers contributing during the month,Male,Central Govt,18-21,210,2021,4
2,New Subscribers contributing during the month,Female,Central Govt,18-21,35,2021,4
3,New Subscribers contributing during the month,Transgender,Central Govt,18-21,-,2021,4
4,New Subscribers contributing during the month,Non-IRA,Central Govt,18-21,-,2021,4
...,...,...,...,...,...,...,...
97,New Subscribers contributing during the month,Female,Non-Govt (Corporate Sector),non-ira,-,2022,3
98,New Subscribers contributing during the month,Transgender,Non-Govt (Corporate Sector),non-ira,-,2022,3
99,New Subscribers contributing during the month,Non-IRA,Non-Govt (Corporate Sector),non-ira,-,2022,3
100,New Subscribers contributing during the month,Total,Non-Govt (Corporate Sector),non-ira,-,2022,3


In [245]:
fnl_df = pd.concat([fnl_nps, fnl_epf, fnl_esic])

In [246]:
interim_path = Path.cwd().parent / "data" / "interim" / f"{src_pdf_filename}.csv"
fnl_df.to_csv(interim_path, index=False)